In [1]:
import pandas as pd
from example.discretizer.discretizer  import AVDiscretizer, Action
from example.transition import TransitionRecorded
import matplotlib.pyplot as plt
from pathlib import Path
import pgeon.policy_graph as PG
from example.environment import SelfDrivingEnvironment
from nuscenes import NuScenes

In [2]:
#path where you stored the copy of the nuScenes dataset.
DATAROOT = Path('/home/saramontese/Desktop/MasterThesis/example/dataset/data/sets/nuscenes')

#MINI
nuscenes = NuScenes('v1.0-mini', dataroot=DATAROOT)

#FULL
#nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.440 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


The dataset contains the following columns for each state of the self-driving car:

- instance_token: A unique identifier for the vehicle instance.
- translation: The vehicle's position in 3D space (x, y, z coordinates).
- yaw: The orientation of the vehicle around the vertical axis (rotation angle).
- velocity: The vehicle's speed in a given direction.
- acceleration: The change in velocity over time.
- heading_change_rate: The rate of change of the vehicle's direction.
- timestamp: The time at which the state was recorded.
- scene_token: A unique identifier for the scenario or environment the vehicle is in
- detected_objects: detected objects in the surrounding of the vehicle


Load the CSV data into a DataFrame


In [3]:

dtype_dict = {
    'modality': 'category',  # for limited set of modalities, 'category' is efficient
    'scene_token': 'str',  
    'timestamp': 'str',  # To enable datetime operations
    'rotation': 'object',  # Quaternion (lists)
    'x': 'float64',
    'y': 'float64',
    'z': 'float64',
    'yaw': 'float64',  
    'velocity': 'float64',
    'acceleration': 'float64',
    'heading_change_rate': 'float64',
    'delta_local_x': 'float64',
    'delta_local_y': 'float64'}
df = pd.read_csv(DATAROOT / 'train_v1.0-trainval_lidar_0.csv', dtype=dtype_dict, parse_dates=['timestamp'])
#df['detect_CAM_FRONT'] = df['detect_CAM_FRONT'].apply(lambda x: ast.literal_eval(x))
#df['detect_CAM_BACK'] = df['detect_CAM_BACK'].apply(lambda x: ast.literal_eval(x))
df.head()

,scene_token,steering_angle,modality,timestamp,rotation,x,y,z,yaw,velocity,acceleration,delta_local_x,delta_local_y
0,0053e9c440a94c1b84bd9c4223efc4b0,0.179841,lidar,2018-07-27 16:20:30.048445,"[0.2766247744031405, -0.0027369692279543625, 0...",427.968589,1622.155824,0.0,2.581053,0.000018,0.000028,0.000000e+00,0.000000
1,0053e9c440a94c1b84bd9c4223efc4b0,0.179841,lidar,2018-07-27 16:20:30.548859,"[0.27663215946727443, -0.002737971665731827, 0...",427.968593,1622.155822,0.0,2.581038,0.000008,-0.000020,-7.657486e-11,-0.000004
2,0053e9c440a94c1b84bd9c4223efc4b0,0.178096,lidar,2018-07-27 16:20:31.048731,"[0.27662503731230986, -0.002729398219646146, 0...",427.968591,1622.155823,0.0,2.581052,0.000004,-0.000008,7.038325e-10,0.000002
3,0053e9c440a94c1b84bd9c4223efc4b0,0.179841,lidar,2018-07-27 16:20:31.448537,"[0.2766891809143752, -0.0027446222315419917, 0...",427.968585,1622.155827,0.0,2.580919,0.000018,0.000036,9.009808e-10,0.000007
4,0053e9c440a94c1b84bd9c4223efc4b0,0.179841,lidar,2018-07-27 16:20:31.948403,"[0.276690691475819, -0.002853827740243303, 0.0...",427.968578,1622.155832,0.0,2.580916,0.000017,-0.000003,-5.687954e-10,0.000009


## Trajectory of AV in a Scene

Let's do an example of computing trajectory of a vehicle in a scene. We then check if the rendering of the scene match the computed trajectory.

In [4]:
# initialize discretizer
av_discretizer = AVDiscretizer()


List of scenes

In [5]:
df['scene_token'].unique()

array(['0053e9c440a94c1b84bd9c4223efc4b0',
       '00590cbfa24a430a8c274b51e1c71231',
       '01452fbfbf4543af8acdfd3e8a1ee806',
       '01796494e25c448dadaba70cfcc3532d',
       '01c3f5e39956402da3e37845632fadca',
       '01e4fcbe6e49483293ce45727152b36e',
       '0287702725184161af95373198546a59',
       '02f1e5e2fc544798aad223f5ae5e8440',
       '034256c9639044f98da7562ef3de3646',
       '034dee1695304630b0692da8c1f153fc',
       '03a8a37d5d444674bae34336e78ac39d',
       '03ee880dd4e348f4b3407f0d073c7c70',
       '04061484361145f5ac6f90a5103b84a6',
       '04219bfdc9004ba2af16d3079ecc4353',
       '04e5f089805540a5a9e73c1b7c54ee8c',
       '055a607b74e04001adef097225a8661a',
       '05d5241d4d5a4c2a9c670dc8c9853c74',
       '06c43bc74aff453081ce83eb126a756e',
       '078e6d7a3d4a429bba404bf12c7a98f2',
       '080a52cb8f59489b9cddc7b721808088',
       '08110c8640454affb5167801442a85fd',
       '087c6e521e974c708d49cac1db095691',
       '08be42eb2186411d8e2201225329f1c6',
       '095

Select a scene and test the  algorithm


In [6]:
scene_test = df['scene_token'].unique()[0]
example_scene_df = df[df['scene_token']==scene_test]
example_scene_df.head()

,scene_token,steering_angle,modality,timestamp,rotation,x,y,z,yaw,velocity,acceleration,delta_local_x,delta_local_y
0,0053e9c440a94c1b84bd9c4223efc4b0,0.179841,lidar,2018-07-27 16:20:30.048445,"[0.2766247744031405, -0.0027369692279543625, 0...",427.968589,1622.155824,0.0,2.581053,0.000018,0.000028,0.000000e+00,0.000000
1,0053e9c440a94c1b84bd9c4223efc4b0,0.179841,lidar,2018-07-27 16:20:30.548859,"[0.27663215946727443, -0.002737971665731827, 0...",427.968593,1622.155822,0.0,2.581038,0.000008,-0.000020,-7.657486e-11,-0.000004
2,0053e9c440a94c1b84bd9c4223efc4b0,0.178096,lidar,2018-07-27 16:20:31.048731,"[0.27662503731230986, -0.002729398219646146, 0...",427.968591,1622.155823,0.0,2.581052,0.000004,-0.000008,7.038325e-10,0.000002
3,0053e9c440a94c1b84bd9c4223efc4b0,0.179841,lidar,2018-07-27 16:20:31.448537,"[0.2766891809143752, -0.0027446222315419917, 0...",427.968585,1622.155827,0.0,2.580919,0.000018,0.000036,9.009808e-10,0.000007
4,0053e9c440a94c1b84bd9c4223efc4b0,0.179841,lidar,2018-07-27 16:20:31.948403,"[0.276690691475819, -0.002853827740243303, 0.0...",427.968578,1622.155832,0.0,2.580916,0.000017,-0.000003,-5.687954e-10,0.000009


Compute the trajectoy of the scene

In [7]:

import pgeon.policy_graph as PG

recorder = TransitionRecorded()
av_discretizer = AVDiscretizer()
env = SelfDrivingEnvironment()
pg = PG.PolicyGraph(env, av_discretizer)
pg = pg.fit(df, update=False, verbose=False)

Fitting PG from scenes...: 100%|██████████| 668/668 [00:39<00:00, 16.73it/s]

Average Reward: 7.928068862275447 and Standard Deviation: 7.697623982975654 --> Epoch Mean Time: 39.925320863723755


To render the scene (video):

In [8]:
#nuscenes.render_scene(scene_test)

# Policy Graph Generation

In [9]:
print(f'Number of nodes: {len(pg.nodes)}')
print(f'Number of edges: {len(pg.edges)}')

Number of nodes: 12559
Number of edges: 18307


In [10]:
arbitrary_state = list(pg.nodes)[10]

print(arbitrary_state)
print(f'  Times visited: {pg.nodes[arbitrary_state]["frequency"]}')
print(f'  p(s):          {pg.nodes[arbitrary_state]["probability"]:.3f}')

(Position(106,405), Velocity(STOPPED), Rotation(FORWARD))
  Times visited: 45
  p(s):          0.002


In [11]:
arbitrary_edge = list(pg.edges)[66]

print(f'From:    {arbitrary_edge[0]}')
print(f'Action:  {arbitrary_edge[2]}')
print(f'To:      {arbitrary_edge[1]}')
print(f'  Times visited:      {pg[arbitrary_edge[0]][arbitrary_edge[1]][arbitrary_edge[2]]["frequency"]}')
print(f'  p(s_to,a | s_from): {pg[arbitrary_edge[0]][arbitrary_edge[1]][arbitrary_edge[2]]["probability"]:.3f}')

From:    (Position(105,406), Velocity(LOW), Rotation(SLIGHT_RIGHT))
Action:  8
To:      (Position(105,406), Velocity(LOW), Rotation(SLIGHT_RIGHT))
  Times visited:      3
  p(s_to,a | s_from): 0.176


### Using PG-based policies

In [12]:
from pgeon import PGBasedPolicy, PGBasedPolicyMode, PGBasedPolicyNodeNotFoundMode
import numpy as np

In [13]:
policy_mode = PGBasedPolicyMode.STOCHASTIC
node_not_found_mode = PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES

policy = PGBasedPolicy(pg, policy_mode,node_not_found_mode )
pg._normalize()
print(f'Policy Graph with {policy_mode} and {node_not_found_mode}')
print(f'PG number of nodes: {len(policy.pg.nodes)}')
print(f'PG number of edges: {len(policy.pg.edges)}')
        


initial_state = np.array([329.6474941596216, 660.1966888688361, 5.108549775006556, -0.2023862177193762])
data = {
    'CAM_FRONT': "{('vehicle.bus.rigid', 'vehicle.moving'): 1, ('vehicle.construction', 'vehicle.parked'): 1}",
    'CAM_BACK': "{}"#{('vehicle.truck', 'vehicle.moving'): 1, ('vehicle.car', 'vehicle.moving'): 1, ('vehicle.car', 'vehicle.parked'): 2}
        }
objects = pd.Series(data)
   
final_state = np.array([309.1878607795861, 668.6677254778403,  7.946119310024729e-05, -1.2007145498601297])
policy.pg.discretizer.detection_cameras = ['CAM_FRONT', 'CAM_BACK' ]

action = policy.act(initial_state)

action



Policy Graph with PGBasedPolicyMode.STOCHASTIC and PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES
PG number of nodes: 12559
PG number of edges: 18307


7

In [14]:
discretized_obs = policy.pg.discretizer.discretize(initial_state)#, objects)
str_obs = av_discretizer.state_to_str(discretized_obs)
str_obs

'Position(82,165)&Velocity(LOW)&Rotation(FORWARD)'

Question 1

In [15]:
possible_actions = pg.question1(discretized_obs, verbose=True)

I will take one of these actions:
	-> STRAIGHT 	Prob: 100.0 %


Question 2

In [16]:
action_id = AVDiscretizer.get_action_id(Action.TURN_RIGHT)
best_states = pg.question2(action_id, verbose=True)

*********************************
* When do you perform action 3?
*********************************
Most probable in 261 states:
	-> (Position(598,599), Velocity(MEDIUM), Rotation(SLIGHT_RIGHT))
	-> (Position(114,430), Velocity(LOW), Rotation(RIGHT))
	-> (Position(306,433), Velocity(LOW), Rotation(RIGHT))
	-> (Position(449,296), Velocity(MEDIUM), Rotation(SLIGHT_RIGHT))
	-> (Position(596,599), Velocity(MEDIUM), Rotation(SLIGHT_RIGHT))
	-> (Position(460,246), Velocity(LOW), Rotation(SLIGHT_RIGHT))
	-> (Position(601,597), Velocity(MEDIUM), Rotation(SLIGHT_RIGHT))
	-> (Position(424,202), Velocity(LOW), Rotation(SLIGHT_RIGHT))
	-> (Position(212,355), Velocity(MEDIUM), Rotation(SLIGHT_RIGHT))
	-> (Position(160,465), Velocity(MEDIUM), Rotation(SLIGHT_RIGHT))
	-> (Position(459,247), Velocity(LOW), Rotation(SLIGHT_RIGHT))
	-> (Position(631,202), Velocity(LOW), Rotation(SLIGHT_RIGHT))
	-> (Position(488,196), Velocity(LOW), Rotation(SLIGHT_RIGHT))
	-> (Position(230,179), Velocity(MEDIUM), Rotati

Question 3

In [17]:
policy_modes = [PGBasedPolicyMode.STOCHASTIC, PGBasedPolicyMode.GREEDY]
node_not_found_modes = [PGBasedPolicyNodeNotFoundMode.RANDOM_UNIFORM, PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES]
action_id = AVDiscretizer.get_action_id(Action.BRAKE)

str_to_state = av_discretizer.str_to_state(str_obs)

for policy_mode in policy_modes:
    for node_not_found_mode in node_not_found_modes:
        policy = PGBasedPolicy(pg, mode=policy_mode,  node_not_found_mode=node_not_found_mode)
        greedy = True if policy_mode is PGBasedPolicyMode.GREEDY else False
        print(f'Policy Graph with {policy_mode} and {node_not_found_mode}')
        print()
        print(f'Supposing I was in {str_to_state}, '
            f'if I did not choose to BRAKE was due to...')
        policy.pg._normalize() #do i need to normalize before this step?
        counterfactuals = policy.pg.question3(str_to_state, action_id, greedy, verbose=True)
        
        print()

Policy Graph with PGBasedPolicyMode.STOCHASTIC and PGBasedPolicyNodeNotFoundMode.RANDOM_UNIFORM

Supposing I was in (Position(82,165), Velocity(LOW), Rotation(FORWARD)), if I did not choose to BRAKE was due to...
***********************************************
* Why did not you perform X action in Y state?
***********************************************
I would have chosen: 7
I would have chosen 5 under the following conditions:
Hypothetical state: (Position(81,165), Velocity(LOW), Rotation(FORWARD))
   Actual: <class 'example.discretizer.utils.Position'> = Position(82,165) -> Counterfactual: <class 'example.discretizer.utils.Position'> = Position(81,165)

Policy Graph with PGBasedPolicyMode.STOCHASTIC and PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES

Supposing I was in (Position(82,165), Velocity(LOW), Rotation(FORWARD)), if I did not choose to BRAKE was due to...
***********************************************
* Why did not you perform X action in Y state?
***********************

Question 4

1. select a scene
2. plot/render the scene and show the trajectory
3. show the comparison of each action to the other actions

In [18]:
scene_test = df['scene_token'].unique()[0]
example_scene_df = df[df['scene_token']==scene_test]
#example_scene_df

In [19]:
#nuscenes.render_scene(scene_test)

In [20]:
#TODO: consider objects as well
for i in range(len(example_scene_df)-1):
    state = example_scene_df.iloc[i][['delta_local_x', 'delta_local_y', 'velocity', 'acceleration', 'steering_angle']].tolist()
    next_state = example_scene_df.iloc[i+1][['delta_local_x', 'delta_local_y', 'velocity', 'acceleration', 'steering_angle']].tolist()
    action = av_discretizer.determine_action(state, next_state)

    disc_state =  av_discretizer.discretize(example_scene_df.iloc[i][['x', 'y', 'velocity', 'steering_angle']].tolist())
    pg.question4(disc_state, action, verbose=True)

***********************************************
* why the vehicle in state s preferred to take the action a instead of all other possible actions?
***********************************************
For Action.IDLE compared to Action.TURN_LEFT:
Speed Reward Difference: 0
Safety Reward Difference: 0
Smoothness Reward Difference: 0.1
Progress Reward Difference: 0
-------------------------------------------------------
For Action.IDLE compared to Action.TURN_RIGHT:
Speed Reward Difference: 0
Safety Reward Difference: 0
Smoothness Reward Difference: 0.1
Progress Reward Difference: 0
-------------------------------------------------------
For Action.IDLE compared to Action.GAS:
Speed Reward Difference: 0
Safety Reward Difference: 0
Smoothness Reward Difference: 0.0
Progress Reward Difference: 0
-------------------------------------------------------
For Action.IDLE compared to Action.BRAKE:
Speed Reward Difference: 0
Safety Reward Difference: 0
Smoothness Reward Difference: 0.0
Progress Reward 

# Policy Graph Evaluation and Comparison

## Compute Static Metrics


In [21]:
from example.evaluator import PolicyGraphEvaluator
evaluator = PolicyGraphEvaluator(pg)
entropy_metrics_graph = evaluator.compute_entropy_metrics()

for state, metrics in entropy_metrics_graph.items():
    print(f"State: {state}, H(s): {metrics['Hs']:.2f}, Ha(s): {metrics['Ha']:.2f}, Hw(s): {metrics['Hw']:.2f}")

State: (Position(104,407), Velocity(LOW), Rotation(SLIGHT_RIGHT)), H(s): 1.75, Ha(s): 1.00, Hw(s): 0.75
State: (Position(104,410), Velocity(LOW), Rotation(SLIGHT_RIGHT)), H(s): 2.25, Ha(s): 1.38, Hw(s): 0.87
State: (Position(105,414), Velocity(LOW), Rotation(FORWARD)), H(s): 1.50, Ha(s): 1.00, Hw(s): 0.50
State: (Position(106,405), Velocity(LOW), Rotation(SLIGHT_RIGHT)), H(s): 1.94, Ha(s): 0.97, Hw(s): 0.97
State: (Position(104,409), Velocity(LOW), Rotation(SLIGHT_RIGHT)), H(s): 2.29, Ha(s): 1.44, Hw(s): 0.86
State: (Position(104,412), Velocity(LOW), Rotation(FORWARD)), H(s): 0.00, Ha(s): 0.00, Hw(s): 0.00
State: (Position(104,407), Velocity(LOW), Rotation(RIGHT)), H(s): 2.19, Ha(s): 1.30, Hw(s): 0.90
State: (Position(104,411), Velocity(LOW), Rotation(SLIGHT_RIGHT)), H(s): 2.68, Ha(s): 1.45, Hw(s): 1.23
State: (Position(106,406), Velocity(LOW), Rotation(SLIGHT_RIGHT)), H(s): 2.00, Ha(s): 1.50, Hw(s): 0.50
State: (Position(106,405), Velocity(LOW), Rotation(FORWARD)), H(s): 2.43, Ha(s): 

Ideal situation: knowing the current state unqeuivocally determines the following action and state. To evaluate this we use Entropy.

Entropy Interpretation: level of uncertainty of a variable (check PG_position_paper).

- H_a(s): 0 --> state s perfectly determines the following action a.
Too simple graphs with few nodes show larger action uncertainty, making output less reliable.

- H_w(s): 0 --> state s perfectly determines the following state s'.

- H(s): 0 --> state s perfectly determines the following pais of s', a.

## Improve Policy

In [22]:
#path_pg_data = Path('/home/saramontese/Desktop/MasterThesis')

#sub_pg = PG.PolicyGraph.from_nodes_and_edges(str(path_pg_data / 'nuscenes_nodes.csv'), str(path_pg_data / 'nuscenes_edges.csv'), env, av_discretizer  )

In [23]:
#initial_state = example_scene_df.iloc[0][['x', "y", "velocity", "yaw"]].values
#final_state = example_scene_df.iloc[-1][['x', "y", "velocity", "yaw"]].values
#print('initial scene state: ', initial_state)
#print('final scene state: ', final_state)

In [24]:
#sub_agent = PGBasedPolicy(sub_pg, mode=PGBasedPolicyMode.GREEDY)
#reward = env.compute_total_reward(sub_agent, initial_state, final_state, max_steps=100)
#print('scene reward before improvement: ', reward)

In [25]:
#reward = env.compute_total_reward(pg)
#print('Total PG reward before improvement: ', reward)

In [26]:
#pg.policy_iteration(sub_pg)
#print('scene reward after improvement: ', env.compute_total_reward(pg))


## Compute Rewards

1. I consider the total pg.
2. I consider each scene as episode, with a initial state and end state
3. For each scene, i compute the reward of the PG.
4. I compute the avg scene reward.

I do this for all the possible pgBased policies.

in which format the scenes?
can i create a function that creates a pg from a dataframe without having to store it the dataframe as csv and then load it?

In [27]:
#for each scenes:
    #initial_state = example_scene_df.iloc[0][['x', "y", "velocity", "yaw"]].values
    #final_state = example_scene_df.iloc[-1][['x', "y", "velocity", "yaw"]].values
    #print('initial scene state: ', initial_state)
    #print('final scene state: ', final_state)